In [42]:
import pandas as pd
import numpy as np
import cufflinks
import os
import re
import pickle

In [43]:
dataset_class='baseball'
dataset_name='baseball'

N_features = [3, 5, 7, 10, 20, 30, 50, 70, 90]
N_samples = 100

In [44]:
def get_result_data(n_features, dataset_name, key, N_samples, path):
    """
    path: path to folder containing pickle dumps
todo: there is only one best in result data (saving memory). show random and noisy_mean
    :return: {no_features: {error: auc},...} e.g. {16: {0.200036667: 0.53119531952662713, 0.105176567: 0.57273262130177505
    """
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path,f))]

    results = dict()
    pattern = r'(\d+)features_{}samples_(.*?)error'.format(N_samples)
    for f in files:
        match = re.match(pattern, f)
        if match:
            no_features = int(match.group(1))
            error = float(match.group(2))

            if no_features not in results.keys():
                results[no_features] = dict()

            results[no_features][error] = (np.mean(pickle.load(open(os.path.join(path, f), 'rb'))[key]))
    if n_features:
        results = {r:results[r] for r in n_features}
    return results

In [45]:
path_pickle = path='/home/marcello/studies/bachelorarbeit/workspace/github_crowd-sourcing-for-feature-selection/pickle-dumps/baseball/v1'

results_noisy = get_result_data(N_features, dataset_name, key='best_noisy_mean', N_samples=N_samples, path=path_pickle)
results_best = get_result_data(N_features, dataset_name, key='best', N_samples=N_samples, path=path_pickle)
results_rand = get_result_data(N_features, dataset_name, key='random', N_samples=N_samples, path=path_pickle)

In [46]:
df_noisy = pd.DataFrame(results_noisy)
df_best = pd.DataFrame(results_best)
df_rand = pd.DataFrame(results_rand)

In [47]:
df = pd.concat(dict(noisy=df_noisy, best=df_best, random=df_rand), axis='columns')

In [48]:
df.iplot(title='AUC vs. error rate (v1: rank same or higher)', xTitle='Error rate', yTitle='AUC')

In [49]:
N_feat = [5, 10, 20]
df2 = pd.concat(dict(noisy=df_noisy[N_feat], best=df_best[N_feat], random=df_rand[N_feat]), axis='columns')

In [50]:
df2.iplot(title='AUC vs. error rate (v1: rank same or higher)', xTitle='Error rate', yTitle='AUC')